In [1]:
# Import required libraries

import urllib.request, urllib.parse, urllib.error
import bs4 as bs
import ssl
import numpy as np 
import pandas as pd 


Use the Notebook to build the code to scrape the following Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 

In [25]:

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


html = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', context=ctx).read()
soup = bs.BeautifulSoup(html, 'html.parser')

# Retrieve all of the table rows and put them into a pandas dataframe
row_list = []
table_rows = soup('tr')
for row in table_rows:
    td = row.find_all('td')
    row = [tr.text for tr in td]
    row_list.append(row)
Toronto_DF = pd.DataFrame(row_list)



Process the scraped dataframe to satisfy the following requirements:

1- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.  
2- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.  
3- More than one neighborhood can exist in one postal code area.   
4- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  
5- Use the .shape method to print the number of rows of your dataframe.

In [26]:
#Clean up the dataframe

#Just keep required columns
Toronto_DF = Toronto_DF[[0,1,2]]

#Rename columns
Toronto_DF.columns=["PostalCode", "Borough", "Neighborhood"]

#Remove any '\n' in the strings
Toronto_DF = Toronto_DF.replace('\n','', regex=True)

#Drop rows that are not required
Toronto_DF = Toronto_DF.dropna()
Toronto_DF = Toronto_DF[Toronto_DF.Borough != 'Not assigned']
Toronto_DF = Toronto_DF[Toronto_DF.Borough != 'B' ]
Toronto_DF = Toronto_DF[Toronto_DF.Borough != 'NL']
Toronto_DF = Toronto_DF[Toronto_DF.Borough != 'NS']

#Group rows if required
Toronto_DF['Neighborhood'] = Toronto_DF.groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
Toronto_DF = Toronto_DF.drop_duplicates()

Toronto_DF.shape

(103, 3)